In [2]:
import utils.sql as sql_tools
import pandas as pd

In [56]:
cnx, cursor = sql_tools.create_session_cursor()

query = '''
SELECT od.order_id, od.food_id, food_name, food_type, quantity, price, order_time from 
order_details od inner join food inner join `order`
ON od.food_id = food.food_id AND od.order_id = `order`.order_id
WHERE od.order_id in (
	select order_id from `order` where restaurant_id = 1
);
'''

result = sql_tools.execute_fetchall(cursor, query, tuple())
result = pd.DataFrame(result)

result.columns = ['ORDER_ID', 'FOOD_ID', 'FOOD_NAME', 'FOOD_TYPE', 'QUANTITY', 'PRICE', 'ORDER_TIME']
result = result.astype({'QUANTITY': int})
len(result)
result.head()
# result.dtypes

,ORDER_ID,FOOD_ID,FOOD_NAME,FOOD_TYPE,QUANTITY,PRICE,ORDER_TIME
0,1,1,辣椒炒肉,荤菜,1,42.0,2023-03-04 11:49:05
1,1,2,小炒黄牛肉,荤菜,1,68.0,2023-03-04 11:49:05
2,1,3,剁椒鱼头,荤菜,1,96.0,2023-03-04 11:49:05
3,1,5,烟笋炒腊肉,荤菜,1,48.0,2023-03-04 11:49:05
4,1,6,有机花菜,素菜,1,36.0,2023-03-04 11:49:05


In [ ]:
def _process_overturn_analysis(df, viewmode='D'):
    ''' Processing for the overturn analysis. This function receives food order, then:
        1. Calculate the price * quantity
        2. Group by FOOD_TYPE
        3. Group by date (with viewmode)
    '''
    # step 1
    df['Money'] = df['QUANTITY'] * df['PRICE']
    # step 2
    ans = pd.DataFrame()
    df1 = df.groupby('FOOD_TYPE')
    for food_type, sub_df in df1:
        # step 3
        date_grouped = sub_df.groupby(pd.Grouper(key='ORDER_TIME', axis=0, freq=viewmode)).sum(numeric_only=True)
        ans[food_type] = date_grouped['Money']
    ans.index = [ans.index[i].to_pydatetime().strftime('%Y-%m-%d') for i in range(len(ans.index))]
    return ans
_process_overturn_analysis(result, 'D')

In [ ]:
def _get_total_turnover():
    ...

In [67]:
def _process_dish_analysis(df, viewmode='D'):
    ''' Processing for the overturn analysis. This function receives food order,
        Then group by food name and date respectively.
    '''
    ans = pd.DataFrame()
    df1 = df.groupby('FOOD_NAME')
    for food_name, sub_df in df1:
        date_grouped = sub_df.groupby(pd.Grouper(key='ORDER_TIME', axis=0, freq=viewmode)).sum(numeric_only=True)
        ans[food_name] = date_grouped['QUANTITY']
    ans.index = [ans.index[i].to_pydatetime().strftime('%Y-%m-%d') for i in range(len(ans.index))]
    print(list(ans.columns[:2]))
    return ans
_process_dish_analysis(result)[['剁椒鱼头', '小炒黄牛肉']]

['剁椒鱼头', '口味鸡']


,剁椒鱼头,小炒黄牛肉
2023-03-01,22,19
2023-03-02,18,22
2023-03-03,26,29
2023-03-04,17,20
2023-03-05,13,19
2023-03-06,17,23
2023-03-07,13,20
2023-03-08,22,28
2023-03-09,18,27
2023-03-10,14,23


In [18]:
# groupby('FOOD_TYPE').
df1 = result.groupby('FOOD_TYPE')#.sum()
for i in df1:
    print(i)

('汤',       ORDER_ID  FOOD_ID FOOD_NAME FOOD_TYPE  QUANTITY  PRICE  \
6            1        9    茶树菇土鸡汤         汤         1   56.0   
7            1       10     筒骨煨湖藕         汤         1   52.0   
14           2       10     筒骨煨湖藕         汤         1   52.0   
20           3        9    茶树菇土鸡汤         汤         1   56.0   
21           3       10     筒骨煨湖藕         汤         1   52.0   
...        ...      ...       ...       ...       ...    ...   
3572       598        9    茶树菇土鸡汤         汤         1   56.0   
3578       599        9    茶树菇土鸡汤         汤         1   56.0   
3579       599       10     筒骨煨湖藕         汤         1   52.0   
3584       600        9    茶树菇土鸡汤         汤         1   56.0   
3585       600       10     筒骨煨湖藕         汤         1   52.0   

              ORDER_TIME  
6    2023-03-04 11:49:05  
7    2023-03-04 11:49:05  
14   2023-03-15 18:31:36  
20   2023-03-06 18:00:21  
21   2023-03-06 18:00:21  
...                  ...  
3572 2023-03-10 13:00:16  
3578 2023